<a href="https://colab.research.google.com/github/fielia/Election-Script/blob/main/Election_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *EDITABLE VARIABLES*


Please make sure to have the `.csv` file only include one election (captain, lead, etc), with the ranking columns named after the candidates they represent.
&nbsp;  
&nbsp;  

Also, confirm that the ranking values are **exactly** as inputted below (case insensitive) in the order of *highest ranked* to *lowest ranked*.
&nbsp;  
&nbsp;  

If you would like the program to automatically filter out seniors,
change `check_seniors` to `True` and set the column titles for the seniors to end with a character, specified in the variable called `senior_signifier`.
&nbsp;  
&nbsp;  

You can also color-code the candidates! To do so, set `color_names` to `True`. Whether you want color-coded names or not, all names will be bolded and underlined, and other words will be colored to help readability.<br>

*NOTE: There is a limited amount of colors the program can use. If there are more candidates than colors, the program will **NOT** use colors, not matter what `color_names` is set to.*

---

Once the variables are set to your liking, go to the `Toolbar`, select `Runtime`, then choose `Run All`. Then, go to the `Output Here` section to see the results of the election.
&nbsp;  
&nbsp;  

If any errors are encountered that do not state an issue with the `Editable Variables`, please contact **Ritvik Garg** for assistance.

In [13]:
# the location of the voting.csv file in the colab (keep the quotes, parenthesis, and the 'r' in front of the quotes)
path: str = (r"/content/voting2.csv")

# the column the first candidate is at, in numbers (if the first column is D, enter '4')
start_col: int = 4

# the amount of candidates in the vote
num_candidates: int = 3

# the amount of seats to elect for
num_seats: int = 2

# the text in the table for the choices, in order of (highest ranked to lowest ranked) -- this value, both here and in the .csv input, are case insensitive
ranking_names: list = ["fiRSt cHoicE", "sEcOnd chOIcE", "third choice", "FOURTH CHOICE", "Fifth Choice"]

# whether the program's user wants to filter out seniors, and what end character they are signified by, or let the program do so
check_seniors: bool = False
senior_signifier: str = '*' # does not matter if check_seniors is False

# enable colors for the candidate names in the output
color_names: bool = True

## PLEASE <u>*DO NOT*</u> EDIT CODE IN THIS SECTION! <u>**Things might break if you do!**</u>

In [14]:
# class containing ANSI shortcuts for text formatting

class color:
    # PURPLE = '\033[95m'
    # CYAN = '\033[96m'
    # BLUE = '\033[94m'
    # YELLOW = '\033[93m'
    # DARK_CYAN = '\033[36m'
    # DARK_BLUE = '\033[34m'
    COLORS: list = ['\033[95m', '\033[96m', '\033[94m', '\033[93m', '\033[36m', '\033[34m']
    RED: str = '\033[91m'
    GREEN: str = '\033[92m'
    BOLD: str = '\033[1m'
    ITALICS: str = '\033[3m'
    UNDERLINE: str = '\033[4m'
    BLACK_BG: str = '\033[40m'
    END: str = '\033[0m'

In [15]:
import pandas as pd
from random import choice, shuffle
from os.path import exists
from sys import exit

def confirm_inputs() -> None:
    """
    Validates input parameters to ensure they meet the required conditions for the election.
    Exits the program if any of the conditions are not met.
    """
    if len(ranking_names) < num_candidates:
        exit("Something went wrong with the amount of ranking names. There should be as many ranking names as candidates. Please try again.")
    elif num_candidates < num_seats:
        exit("Something went wrong with the amount of seats. There should be fewer seats than candidates. Please try again.")
    elif num_candidates == 1:
        exit("Something went wrong with the amount of candidates. There should be at least two candidates. Please try again.")
    elif num_candidates == num_seats:
        exit("Something went wrong with the amount of candidates. The amount of seats is equal to the amount of candidates, and thus the election does not need to happen.")
    elif not exists(path):
        exit("Something went wrong with the path. Please check that the path exists and try again.")

ranking_names = [name.lower() for name in ranking_names]

def fix_value(ranking: str) -> float:
    """
    Converts a ranking string to its corresponding numerical value based on its position in the ranking_names list.
    Returns None if the ranking is NaN.
    """
    if pd.isna(ranking):
        return None
    ranking = ranking.lower()
    for i in range(len(ranking_names)):
        if ranking_names[i] == ranking:
            return i + 1
    return None

def get_quota(num_votes: int) -> float:
    """
    Calculates the quota required to win a seat in the election based on the number of votes and seats.
    """
    return int(float(num_votes) / (num_seats + 1) * 100) / 100.0

def break_tie(candidates: list) -> str:
    """
    Randomly selects a candidate from the list in case of a tie.
    """
    return choice(candidates)

def set_colors(candidates: list) -> list:
    """
    Assigns colors to the candidates for display purposes.
    """
    shuffle(color.COLORS)
    for i in range(len(candidates)):
        if color_names and len(candidates) <= len(color.COLORS):
            candidates[i] = color.ITALICS + color.UNDERLINE + color.COLORS[i] + candidates[i] + color.END
        else:
            candidates[i] = color.ITALICS + color.UNDERLINE + candidates[i] + color.END
    return candidates

def allocate_first_votes(df: pd.DataFrame) -> dict:
    """
    Allocates the first preference votes to each candidate.
    """
    votes: dict = {}
    for index, vote in df.iterrows():
        candidate = vote.idxmin() if not vote.isna().all() else None
        if candidate is not None:
            if candidate not in votes:
                votes[candidate] = []
            votes[candidate].append(vote)
    return votes

def set_candidate_color(candidate: str, candidates: list) -> str:
    """
    Returns the candidate's name with its assigned color.
    """
    for i in range(len(candidates)):
        if color_names and len(candidates) <= len(color.COLORS):
            if candidates[i][13:-4] == candidate:
                return '\033[40m' + candidates[i]
        else:
            if candidates[i][8:-4] == candidate:
                return candidates[i]
    return None

def set_scores(vote_assign: dict) -> dict:
    """
    Calculates the scores for each candidate based on the votes assigned to them.
    """
    scores: dict = {}
    for candidate in vote_assign:
        scores[candidate] = 0.0
        for vote in vote_assign[candidate]:
            vote_val = vote[candidate]
            scores[candidate] += pow(vote_val, -1)
    return scores

def print_round_results(victors: list, candidates: list, candidate_scores: dict, round_num: int) -> None:
    """
    Prints the results of each round of voting.
    """
    print(color.BOLD + "Round " + str(round_num) + ":" + color.END)
    for candidate in candidate_scores:
        candidate_score = candidate_scores[candidate]
        if candidate in victors:
            candidate = set_candidate_color(candidate, candidates)
            print("\t" + candidate + " has " + color.BOLD + color.GREEN + "won" + color.END + " with " + str(candidate_score) + " points!")
        else:
            candidate = set_candidate_color(candidate, candidates)
            print("\t" + candidate + " has " + color.BOLD + color.RED + "not won" + color.END + " with " + str(candidate_score) + " points.")
    print("\n")

def add_to_victors(quota: int, candidate_scores: dict, num_current_victors: int, num_seats: int) -> list:
    """
    Adds candidates who meet the quota to the list of victors.
    """
    new_victors: list = []
    for candidate in candidate_scores:
        if candidate_scores[candidate] >= quota:
            new_victors.append(candidate)
    if len(new_victors) > num_seats - num_current_victors:
        return remove_worst_victors(new_victors, candidate_scores, len(new_victors) - num_seats + num_current_victors)
    return new_victors

def remove_worst_victors(new_victors: list, scores: dict, num_remove: int) -> list:
    """
    Removes the worst-performing candidates from the list of new victors if there are too many.
    """
    min_score: int = scores[new_victors[0]]
    min_candidates: list = []
    for candidate in new_victors:
        if scores[candidate] == min_score:
            min_candidates.append(candidate)
        elif scores[candidate] < min_score:
            min_candidates = [candidate]
            min_score = scores[candidate]
    if len(min_candidates) > 0 and len(min_candidates) < num_remove:
        new_victors = [i for i in new_victors if i not in min_candidates]
        return remove_worst_victors(new_victors, scores, num_remove - len(min_candidates))
    elif len(min_candidates) > num_remove:
        while len(min_candidates) > num_remove:
            removed = break_tie(min_candidates)
            min_candidates.remove(removed)
    new_victors = [i for i in new_victors if i not in min_candidates]
    return new_victors

def add_victors(current_victors: list, candidates: list, candidate_scores: dict, quota: int, round_num: int) -> dict:
    """
    Adds new victors to the current list of victors and prints the round results.
    """
    new_victors = add_to_victors(quota, candidate_scores, len(current_victors), num_seats)
    print_round_results(new_victors, candidates, candidate_scores, round_num)
    return new_victors

def get_surplus(quota: int, candidate_score: int) -> float:
    """
    Calculates the surplus votes for a candidate who exceeds the quota.
    """
    return float(candidate_score - quota) / candidate_score

def reassign_votes(new_victors: list, victors: list, vote_assign: dict, quota: int, candidate_scores: dict) -> dict:
    """
    Reassigns the surplus votes of new victors to other candidates.
    """
    for candidate in new_victors:
        surplus_fraction = get_surplus(quota, candidate_scores[candidate])
        if surplus_fraction == 0.0:
            del vote_assign[candidate]
            continue
        for vote in vote_assign[candidate]:
            vote = vote.apply(lambda x: x * surplus_fraction)

            vote_val = vote[candidate]
            vote_val = vote[vote > vote_val].min()
            new_candidate = vote[vote == vote_val].index[0]
            while new_candidate in victors:
                vote_val = vote[vote > vote_val].min()
                new_candidate = vote[vote == vote_val].index[0]
            if new_candidate not in vote_assign:
                vote_assign[new_candidate] = []
            vote_assign[new_candidate].append(vote)
        del vote_assign[candidate]
    return vote_assign

def remove_last_senior(victors: list, candidates: list, candidate_scores: dict) -> list:
    """
    Replaces the last senior victor if there are too many seniors elected.
    """
    to_replace = victors[-1]
    print("Due to there being too many seniors elected, " + set_candidate_color(to_replace, candidates).replace("*", "") + " is being replaced.\n")
    runner_up = ""
    score = 0
    for candidate in candidate_scores:
        if candidate[-1] != senior_signifier and candidate_scores[candidate] > score:
            score = candidate_scores[candidate]
            runner_up = candidate
    if score == 0:
        print("There are no non-seniors in the pool. Thus, there will be no change to the final candidates.")
    else:
        print(set_candidate_color(runner_up, candidates) + " is replacing them.")
        del victors[-1]
        victors.append(runner_up)
    return victors

def print_winners(victors: list, candidates: list) -> None:
    """
    Prints the final list of elected candidates.
    """
    to_print = ""
    if len(victors) == 1:
        to_print += "\n\nThe winner of the election has been decided!\n"
    else:
        to_print += "\n\nThe winners of the election have been decided!\n"
    for i in range(len(victors)):
        victor = victors[i]
        to_print += set_candidate_color(victor, candidates).replace("*", "")
        if len(victors) == 2 and i == 0:
            to_print += " and "
        elif len(victors) > 2 and i == len(victors) - 2:
            to_print += ", and "
        elif len(victors) > 2 and i != len(victors) - 1:
            to_print += ", "

    if len(victors) == 1:
        to_print += " has been elected! Congratulations!"
    else:
        to_print += " have been elected! Congratulations!"

    print(to_print)

def check_if_over(victors: list, num_seats: int, candidates: list, candidate_scores: dict) -> bool:
    """
    Checks if the election process is over by verifying the number of victors and seniority conditions.
    """
    if len(victors) > num_seats:
        exit("Something went wrong in the program. Contact Ritvik Garg with the following error message: 'More victors than seats.'")
    elif len(victors) < num_seats:
        return False
    if not check_seniors:
        return True
    for victor in victors:
        if victor[-1] != senior_signifier:
            return True
    victors = remove_last_senior(victors, candidates, candidate_scores)
    return True

def set_df() -> pd.DataFrame:
    """
    Loads the election data from a CSV file and prepares the DataFrame for processing.
    """
    global start_col
    start_col -= 1
    confirm_inputs()
    df = pd.read_csv(path)
    df = df.iloc[:, start_col:(num_candidates + start_col)]
    df = df.reset_index()
    df.index = df.index + 1
    df = df.drop(columns=['index'])
    for col in df:
        df[col] = df[col].apply(fix_value)

    return df

def run_election() -> None:
    """
    Main function to run the election process.
    """
    df = set_df()
    num_votes = df.shape[0]
    candidates = set_colors(list(df.columns))
    quota = get_quota(num_votes)
    vote_assign = allocate_first_votes(df)
    victors = []
    round_num = 0
    while True:
        round_num += 1
        candidate_scores = set_scores(vote_assign)
        new_victors = add_victors(victors, candidates, candidate_scores, quota, round_num)
        victors += new_victors
        if check_if_over(victors, num_seats, candidates, candidate_scores):
            print_winners(victors, candidates)
            return
        vote_assign = reassign_votes(new_victors, victors, vote_assign, quota, candidate_scores)


## *OUTPUT HERE*

In [16]:
## This is it! ##

run_election()

Round 1:
	Niels Greiner has not won with 9.0 points.
	Arvino Mahadi has not won with 10.0 points.
	Jonathan Peng has won with 17.0 points!


Round 2:
	Niels Greiner has not won with 17.499999999999996 points.
	Arvino Mahadi has won with 30.39999999999999 points!




The winners of the election have been decided!
Jonathan Peng and Arvino Mahadi have been elected! Congratulations!
